# 📊 02 - Pré-Processamento dos Dados

## 0. Configuração Inicial

In [2]:
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


In [3]:
df = pd.read_csv('../data/raw/creditcard.csv')
print("✅ Dataset carregado com sucesso!")
df.head()

✅ Dataset carregado com sucesso!


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## 1. Tratamento de duplicatas

In [4]:
print(f"Tamanho antes de remover duplicatas: {df.shape[0]}")
df.drop_duplicates(inplace=True)
print(f"Tamanho após remover duplicatas: {df.shape[0]}")

Tamanho antes de remover duplicatas: 284807
Tamanho após remover duplicatas: 283726


## 2. Escalonamento

In [ ]:
import numpy as np

# 1. Tratamento do Amount (Mantemos o RobustScaler pois é útil para valores monetários com outliers)
scaler = RobustScaler()
df['scaled_Amount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))

# 2. Tratamento Cíclico do Tempo (Time Engineering)
# Primeiro, convertemos o tempo acumulado (segundos) em segundos do dia (0 a 86400)
# Assumindo que o dataset começa em t=0
seconds_in_day = 24 * 60 * 60
df['Time_Day'] = df['Time'] % seconds_in_day

# Criamos as features cíclicas (Seno e Cosseno)
# Isso preserva a proximidade entre 23:59 e 00:01
df['Time_Sin'] = np.sin(2 * np.pi * df['Time_Day'] / seconds_in_day)
df['Time_Cos'] = np.cos(2 * np.pi * df['Time_Day'] / seconds_in_day)

# Removemos as colunas originais e auxiliares, mantendo apenas as transformadas
df.drop(['Time', 'Amount', 'Time_Day'], axis=1, inplace=True)

# Mover as novas colunas para o início para manter o padrão visual
# Lembre-se: Time_Sin e Time_Cos agora representam o horário
cols = ['scaled_Amount', 'Time_Sin', 'Time_Cos'] + [c for c in df.columns if c not in ['scaled_Amount', 'Time_Sin', 'Time_Cos']]
df = df[cols]

print("✅ Engenharia de features temporais (Sin/Cos) aplicada com sucesso!")
df.head()

,scaled_Amount,scaled_Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.774718,-0.995290,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.268530,-0.995290,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.959811,-0.995279,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.411487,-0.995279,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.667362,-0.995267,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


## 3. Distribuição e Balanceamento

In [6]:
normal_df = df[df['Class'] == 0]
fraud_df = df[df['Class'] == 1]

In [7]:
X_train, X_test_normal = train_test_split(
    normal_df, test_size=0.2, random_state=42
)

X_test = pd.concat([X_test_normal, fraud_df])

In [8]:
y_test = X_test['Class']
X_train = X_train.drop('Class', axis=1)
X_test = X_test.drop('Class', axis=1)

## 4. Salvando os dados de treino e teste

In [9]:
X_train.to_csv('../data/processed/creditcard_train.csv', index=False)
X_test.to_csv('../data/processed/creditcard_test.csv', index=False)
y_test.to_csv('../data/processed/creditcard_test_label.csv', index=False)

print("✅ Arquivos salvos com sucesso!")

✅ Arquivos salvos com sucesso!
